In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pandas_profiling import ProfileReport
import seaborn as sns
import math
import datatable as dt
from datatable import sort, f, rowall

# Import dataframes

### Posts

In [ ]:
%%time

posts_datatable = dt.fread("instagram_posts.csv")

In [ ]:
posts_datatable

In [ ]:
%%time

posts=posts_datatable.to_pandas()
posts

### Profiles

In [ ]:
profiles = pd.read_csv('instagram_profiles.csv', delimiter='\t')
profiles.head()

### Locations

In [ ]:
locations= profiles = pd.read_csv('instagram_locations.csv', delimiter='\t')
locations.head()

# [RQ1] After collecting information, the Data Scientists have to know what dataset they are dealing with, so let's start with an Exploratory Data Analysis (EDA). What can you say about our datasets? Please summarise its main characteristics with visual and tabular methods.

# [RQ2] Let's explore the dataset by finding simple insights regarding the profile and posts.

### Plot the number of posts for each profile in descending order.


In [ ]:
profiles2 = profiles[['profile_name','n_posts']]

In [ ]:
profiles2

In [ ]:
#Let's check for NA values
profiles2['n_posts'].isnull().values.any()

A lot, we must manage them

In [ ]:
#Let's drop all na values
profiles2.dropna(inplace=True)

In [ ]:
# Let's check NAs again
profiles2['n_posts'].isnull().values.any()

In [ ]:
profiles2

Good, we can continue now

In [ ]:
profiles2=profiles2.sort_values('n_posts', ascending=False)

In [ ]:
profiles2

In [ ]:
count_post=profiles2.groupby('n_posts').count()
count_post

In [ ]:
count_post.iloc[1:].sum()

In [ ]:
h=profiles2['n_posts']
plt.hist(h,log=True)
plt.show()

##### What can we say about this result?

As we can see, the majority of the instagram profiles have no posts, meanwhile only a small proportion have posted more than 125.000 posts.

### What posts have the highest number of "likes"?

This is the top 10 of most liked posts

In [ ]:
%%time

df=posts[['post_id','numbr_likes']].sort_values(by='numbr_likes', ascending=False).head(10)

df.index=['Post '+str(i+1) for i in range(10)]

df

### What posts have the most and the least number of comments?

In [ ]:
%%time

comments=posts[['post_id','number_comments']]

top_comm=comments.sort_values(by='number_comments', ascending=False)
least_comm=comments.sort_values(by='number_comments', ascending=True)

In [ ]:
#This is the top 10 of posts which have the most number of comments
top_comm.head(10)

In [ ]:
#These are the 10 posts which have the least number of comments
least_comm.head(10)


#There are many posts with zero comment, these are only ten of them, randomly

### How many posts include tagged locations, and how many do not? Show it using an appropriate chart and comment your results.

In [ ]:
posts2=posts[['location_id','post_id']]

yes_location=len(posts2[np.isnan(posts2['location_id']) == False])
yes_location

In [ ]:
no_location=len(posts2[np.isnan(posts2['location_id']) == True])
no_location

In [ ]:
#Showing the result using a pie chart, which is very usefull to catch the difference between the 
total_posts=yes_location+no_location
yes_perc='{:.1%}'.format(yes_location/total_posts)
no_perc='{:.1%}'.format(no_location/total_posts)
myexplode=[0.0,0.1]

#The plots
fig, axs = plt.subplots(1,2)
axs[0].pie([yes_location,no_location], labels=[yes_perc,no_perc], 
        labeldistance=1.15, colors=['steelblue','chocolate'], 
           explode=myexplode)
axs[1].pie([yes_location,no_location], labels=['{:,}'.format(yes_location),'{:,}'.format(no_location)], 
        labeldistance=1.15, colors=['steelblue','chocolate'], explode=myexplode)
plt.legend(['yes_location','no_location'], bbox_to_anchor=(0.85,1.025), loc="upper left")

# Some cool features
fig.tight_layout(pad=2.0)
fig.set_figheight(15)
fig.set_figwidth(15)
plt.show() #remove the annoying text

#### What did we get?

As we can see, the pie chart hightlights how small is the percentage of posts that do not include a tagged position. On the other side, the majority of the analyzed posts include tagged location.

### How many posts include only photos? How many also have videos?

In [ ]:
%%time 

total=posts[['post_type','post_id']].groupby('post_type').count()

photos=int(total.values[0])

photos_and_videos=int(total.values[2])

total

In [ ]:
print('The posts that include only photos are: ','{:,}'.format(photos))

In [ ]:
print('The posts that include both photos and videos are: ','{:,}'.format(photos_and_videos))

### What's the percentage of business accounts vs non-business? What can you interpret regarding that percentage?

In [ ]:
profiles2 = profiles[['is_business_account','profile_name']]

In [ ]:
# Let's give a look to the dataset
profiles2

In [ ]:
#We can already see some NaN, let's do a proper check 
profiles['is_business_account'].isnull().values.any()

In order to find a meaningfull percentage of business accounts vs non-business, we drop all the NaN values

In [ ]:
profiles2.dropna(inplace=True)

In [ ]:
#Now let's check again if it worked
profiles2['is_business_account'].isnull().values.any()

In [ ]:
# Cool, now let's count how many business account
Business_account=len(profiles2[profiles2['is_business_account'] == True])
Business_account

In [ ]:
# Now let's count how many NOT business account
Not_business_account= len(profiles[profiles['is_business_account'] == False])
Not_business_account

In [ ]:
#Now let's give a look at the percentage of Business Account on the total amount of accounts
'{:.1%}'.format(Business_account/(Business_account+Not_business_account))

In [ ]:
#Showing the result using a pie chart, which is very usefull to catch the difference between the 
total_account=Business_account+Not_business_account
yes_perc='{:.1%}'.format(Business_account/total_account)
no_perc='{:.1%}'.format(Not_business_account/total_account)
myexplode=[0.0,0.1]

#The plots
fig, axs = plt.subplots(1,2, figsize=(8,8))
axs[0].pie([Business_account,Not_business_account], labels=[yes_perc,no_perc], 
        labeldistance=1.15, colors=['steelblue','chocolate'], 
           explode=myexplode)
axs[1].pie([Business_account,Not_business_account], labels=['{:,}'.format(Business_account),'{:,}'.format(Not_business_account)], 
        labeldistance=1.15, colors=['steelblue','chocolate'], explode=myexplode)
plt.legend(['Business_account','Not_business_account'], bbox_to_anchor=(1.25,1.125), loc="upper left")

# Some cool features
fig.tight_layout(pad=2.0)
fig.set_figheight(15)
fig.set_figwidth(15)
plt.show() #remove the annoying text

Since the percentage of business account on the total account is 24.8%, it means that for 3 non-business account there is a business account on our dataset. 

# [RQ3] Now it's important to understand the most common times in which users publish their posts

### What is the most common time in which users publish their posts?

### Create a function that receives a time intervals list as a parameter and returns a plot with the number of posts for each given interval.

### Use the function that you created in the previous literal to plot the number of posts between the following time intervals:
| Initial time | Final time   |
|--------------|--------------|
|   06:00:00   |    10:59:59  |
|   11:00:00   |    13:59:59  |
|   14:00:00   |    16:59:59  |
|   17:00:00   |    19:59:59  |
|   20:00:00   |    23:59:59  |
|   00:00:00   |    02:59:59  |
|   03:00:00   |    05:59:59  |



# [RQ4] In most cases, we will not have a consistent dataset, and the one we are dealing with is not an exception (ex. in the given datasets, you may not find the information of the profiles for some of the posts). So let’s enhance our analysis.

### Write a function that, given a profile_id, will be able to return the posts that belong to the given profile_id.

In [ ]:
def searchpost(profile_id):
    df=posts[posts.profile_id == profile_id]
    return df[['post_id','profile_id','numbr_likes','number_comments']]

In [ ]:
searchpost(profile_id=3.0)

### Write another function that, given an input n (an integer), will return the posts that belong to the n top posted profiles (top n profiles that have posted the highest number of posts) that their data is available in the profile.csv using the previously written function.

In [ ]:
def top_posts(n):
    df=pd.DataFrame(columns=['profile_id','post_id'])
    top=profiles.sort_values(by='n_posts', ascending=False).head(n)
    for profile_id in top.profile_id.values:
        df=pd.concat([df,searchpost(profile_id)])
    return df

In [ ]:
top_posts(2)

### What is the average number of "likes" and comments of the top 10 profiles with the highest number of posts which their information is available in profile.csv?

In [ ]:
top_ten=top_posts(10)
top_ten

In [ ]:
#Let's see the distribution of likes 
plt.hist(top_ten['numbr_likes'])
plt.show()

In [ ]:
#The average of "likes" of the top 10 profiles with the highest number of posts
top_ten['numbr_likes'].mean()

In [ ]:
#Let's check the median in order to know moe about our distribution
top_ten['numbr_likes'].median()

Since the median (2.0) is smaller than the average (3.17), it means that our distribution is asymmetric due to some posts which have a huge amount of likes, as matter of fact, the mean is a statistic influenced by exteme observations. On the other hand, the median is a **robust statistic** because it is not affected by outliers. In our case, we can see that half of the posts have less than 2 likes, while the other half have more than 2 likes.

In [ ]:
#Let's see the distribution of likes 
plt.hist(top_ten['number_comments'])
plt.show()

In [ ]:
#The average of "comments" of the top 10 profiles with the highest number of posts
top_ten['number_comments'].mean()

In [ ]:
top_ten['number_comments'].median()

Taking into account the comments of the top 10 profiles with the highest number of posts, we can see how, on average, the posts show no comments, except for some posts which have less than 3. 

### Plot the number of posts that these top 10 profiles have sent on Instagram in the given interval in question RQ3. Interpret the resulting chart.

# [RQ5] The most influential users are the ones with the highest number of “followers", you can now look more into their activity.

### Plot the top 10 most popular users in terms of followers and their number of posts.

In [ ]:
profiles2 = profiles[['profile_name','n_posts', 'followers']]

In [ ]:
#Let's check for NaN values in 'n_posts'
profiles2['n_posts'].isnull().sum()

In [ ]:
#Let's check for NaN values in 'follower'
profiles2['followers'].isnull().sum()

In [ ]:
#Let's drop them
profiles2.dropna(inplace=True)

In [ ]:
#Verify it
profiles2['n_posts'].isnull().sum() == profiles2['followers'].isnull().sum() == 0

In [ ]:
#Let's sort profiles in ascending order by followers and n_posts
top_profiles=profiles2.sort_values(by=['followers','n_posts'], ascending=False).head(10)
top_profiles

In [ ]:
#Now let's plot it
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.set_xlabel('Profile name')
# plot line chart on axis #1
ax1.bar(top_profiles['profile_name'], top_profiles['followers'], width=0.5, alpha=0.5, color='Blue')
ax1.set_ylabel('Number of followers')
plt.xticks(top_profiles['profile_name'], top_profiles['profile_name'], rotation='vertical')
#ax1.set_ylim(0, 25)
# set up the 2nd axis
ax2 = ax1.twinx()

# plot bar chart on axis #2
ax2.bar(top_profiles['profile_name'], top_profiles['n_posts'], width=0.5, alpha=0.5, color='darkgreen', align='edge')
ax2.grid(False) # turn off grid #2
ax2.set_ylabel('Number of posts')
#ax2.set_ylim(0, 90)
fig.legend(['Followers','Posts'], loc="upper center",bbox_to_anchor=(0.5, 0.3, 0.65, 0.5),fontsize='large')
plt.show()

### Who is the most influential user?

In [ ]:
profiles2 = profiles[['profile_name', 'followers', 'profile_id']]

In [ ]:
#Let's check for NaN values in 'n_posts'
profiles2['followers'].isnull().sum()

In [ ]:
#Let's drop them
profiles2.dropna(inplace=True)

In [ ]:
#Let's check for NaN values in 'follower' again
profiles2['followers'].isnull().sum()

In [ ]:
#Verify it
profiles2['followers'].isnull().sum() == 0

In [ ]:
#Let's sort
sorted_profile=profiles2.sort_values(by='followers', ascending=False)

In [ ]:
# Let's see the result
print('The most influential user is "',sorted_profile.head(1)['profile_name'].values[0], 
      '" with ','{:,}'.format(int(sorted_profile.head(1)['followers'])), ' followers')

### Have they posted anything with tagged locations? Extract the most frequent areas on their posts and plot the number of times each city has been visited.

In [ ]:
locations.head()

In [ ]:
#Saving the most influential user's ID
top_user_id=float(sorted_profile.head(1)['profile_id'])
top_user_id

In [ ]:
#Applying the function we already created to find all the posts of such ID
posts_of_top_id=searchpost(top_user_id)

In [ ]:
posts_of_top_id.head()

In [ ]:
list_of_locations=[]
df_posts_of_most_influential=pd.DataFrame(columns=['post_id','location_id'])
for id_top in tqdm(posts_of_top_id['post_id']):
    df_posts_of_most_influential=pd.concat([df,posts[posts.post_id == id_top]])

In [ ]:
df_posts_of_most_influential.head(10)

In [ ]:
#Find locations and plot it


### How many pictures-only posts have they published? How many reels? (only videos) and how many with both contents? Provide the number as percentages and interpret those figures.

In [ ]:
#We already have:
posts_of_top_id

In [ ]:
df=pd.DataFrame(columns=['post_id','post_type'])
for id_top in posts_of_top_id['post_id'][:5]:
    for chunk in tqdm(pd.read_csv('/Users/the_redd_/Desktop/ADM/instagram_posts.csv', 
                                    usecols=['post_id','location_id'], delimiter='\t', chunksize=1000000)):
        df=pd.concat([df,chunk[chunk.post_id == id_top]])
        

In [ ]:
df

### How many "likes" and comments did posts with only pictures receive? How about videos and mixed posts? Try to provide the average numbers and confront them with their followers amount, explaining what you can say from that comparison.

# [RQ6] It's time to get information from the user posting effectiveness.

### What is the average time (days and minutes) a user lets pass before publishing another post? Plot the top 3 users that publish posts more frequently (calculate the average time that passes between posts), including their amount of followers and following. Provide insights from that chart.


### Using the function you previously coded, plot the time intervals with the highest average number of “likes” and the ones with the highest average number of comments on posts.

# [RQ7] Of course, calculating probabilities is a job that any Data Scientist must know. So let's compute some engaging figures

### What's the probability that a post receives more than 20% "likes" of the number of followers a user has?

### Do users usually return to locations? Extract the probability that a user returns to a site after having posted it in the past. Does that probability make sense to you? Explain why or why not.

# [RQ8] Every decision you take in a data-based environment should be reinforced with charts, statistical tests and analysis methods to check whether a hypothesis is correct or not.

### Does more “likes” also mean more comments? Plot a scatter plot of “likes” vs comments for posts?

### Can you find any significant relationship between the time a user publishes a post and the number of comments and “likes”? Use an appropriate statistical test or technique and support your choice.

### What’s the distribution of followers? Plot the empirical distribution of followers amongst all users and extract the mean, mode, and quantiles. Interpret those figures.

### What are histograms, bar plots, scatterplots and pie charts used for?

### What insights can you extract from a Box Plot?